In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
def generate_power_spectra_data(savepaths, system_filepaths):
    '''
    Creates mulitple .csv files with power spectra data corresponding to h, k, p, and q vector quantities 
    of each planet in each system.

    Params:
    - savepath: The path to the .csv file to save the data into.
    - system_filepaths: A list of filepaths that link to the .csv data files where the data is taken from.
    '''
    
    # Validates the input.
    if type(system_filepaths) != list:
        raise Exception('Please input the system filepaths as a list. The input is not a list.')

    # Guards against index errors.
    assert len(savepaths) == len(system_filepaths)

    # Loops through each system filepath in the filepaths list.
    for index in range(len(system_filepaths)):
        system_filepath = system_filepaths[index]
        savepath = savepaths[index]

        try:
            # Reads in the data for the current system as a pandas dataframe.
            data = pd.read_csv(system_filepath)

            # Gets the time series.
            time = data['Time']

            timestep = time[1] - time[0]
            frequencies = np.fft.rfftfreq(n = len(time), d = timestep)

            periods = (1.0 / frequencies)

            # Captures the last timestep of the integration.
            endtime = time.iloc[-1].astype(float)

            # Skips the system if it failed to integrate the full 1 Myr interval.
            if endtime < 1e6:
                continue

            # Gets the relevant columns for all four relevant orbital elements.
            inclination_columns = sorted([column for column in data.columns if 'inclination' in column])
            eccentricity_columns = sorted([column for column in data.columns if 'ecc' in column])
            omega_columns = sorted([column for column in data.columns if 'omega' in column])
            Omega_columns = sorted([column for column in data.columns if 'Omega' in column])

            # Ensures that the arrays are of identical size.
            assert len(inclination_columns) == len(eccentricity_columns) == len(omega_columns) == len(Omega_columns)

            # Calculates the multiplicity of the system from the length of the inclination columns.
            multiplicity = len(inclination_columns)

            # Defines the csv file object.
            csv_file = open(savepath, 'w', encoding = 'utf-8')

            # Initializes the header variable.
            header = 'Period'

            # Generates the header: Time,Planet 1 h,Planet 1 k,Planet 1 p,Planet 2 h...
            for planet_num in range(1, multiplicity + 1):
                planet_label = 'Planet {n}'.format(n = planet_num)
                header += ',' + planet_label + ' h fft,' + planet_label + ' k fft,' + planet_label + ' p fft,' + planet_label + ' q fft'

            # Ends the header line.
            header += '\n'

            # Writes the header to the .csv file.
            csv_file.write(header)

            # Initializes a set of four vector lists of dimension (multiplicity, 1e6).
            h_ffts = np.empty((multiplicity, 5001))
            k_ffts = np.empty((multiplicity, 5001))
            p_ffts = np.empty((multiplicity, 5001))
            q_ffts = np.empty((multiplicity, 5001))

            # Loops through all possible number of planets in a system (1-9 Planets).
            for planet_num in range(multiplicity):
                # Sets the column label.
                inclination_label = inclination_columns[planet_num]
                eccentricity_label = eccentricity_columns[planet_num]
                omega_label = omega_columns[planet_num]
                Omega_label = Omega_columns[planet_num]

                # Gathers the four relevant orbital element values, casting from string to float values.
                inclinations = data[inclination_label].astype(float)
                eccentricities = data[eccentricity_label].astype(float)
                omegas = data[omega_label].astype(float)
                Omegas = data[Omega_label].astype(float)

                # Calculates the vectors using the numpy library.
                h_fft = abs(np.fft.rfft(eccentricities * np.sin(omegas + Omegas)))
                k_fft = abs(np.fft.rfft(eccentricities * np.cos(omegas + Omegas)))
                p_fft = abs(np.fft.rfft(np.sin(inclinations) * np.sin(Omegas)))
                q_fft = abs(np.fft.rfft(np.sin(inclinations) * np.cos(Omegas)))

                h_ffts[planet_num] = h_fft
                k_ffts[planet_num] = k_fft
                p_ffts[planet_num] = p_fft
                q_ffts[planet_num] = q_fft

                # Guards against index errors.
                assert len(h_ffts[planet_num]) == len(k_ffts[planet_num]) == len(p_ffts[planet_num]) == len(q_ffts[planet_num])

            h_ffts = h_ffts.T
            k_ffts = k_ffts.T
            p_ffts = p_ffts.T
            q_ffts = q_ffts.T

            # Loops through each planet in the given system.
            for row_num in range(len(h_ffts)):
                # Initializes the row output.
                row = str()

                # Loops through each column of the given row.
                for col_num in range(len(h_ffts[row_num])):
                    h_str = h_ffts[row_num][col_num].astype(str)
                    k_str = k_ffts[row_num][col_num].astype(str)
                    p_str = p_ffts[row_num][col_num].astype(str)
                    q_str = q_ffts[row_num][col_num].astype(str)

                    if col_num == 0:
                        # Gets the value of each vector at the given row index, casting to strings.
                        periods_str = periods[row_num].astype(str)

                        row += periods_str + ','
                    else:
                        row += ','

                    # Generates the output string for the current row.
                    row += ','.join([h_str, k_str, p_str, q_str])

                row += '\n'

                # Writes the row to the .csv file.
                csv_file.write(row)

        # Handles exceptions.
        except FileNotFoundError as e:
            print(e)

In [10]:
def generate_test_data(savepaths, values, multiplicity):    
    # Validates the input.
    if type(savepaths) != list:
        raise Exception('Please input the system filepaths as a list. The input is not a list.')

    # Guards against index errors.
    assert len(savepaths) == len(values)

    # Loops through each system filepath in the filepaths list.
    for index in range(len(savepaths)):
        savepath = savepaths[index]
        value = values[index]

        try:
            # Reads in the data for the current system as a pandas dataframe.
            data = np.zeros(5001) + value

            time_data = pd.read_csv(os.path.join(os.pardir, os.pardir, 'data', 'raw', 'sysSim_0_randomO.csv'))

            # Gets the time series.
            time = time_data['Time']

            timestep = time[1] - time[0]
            frequencies = np.fft.rfftfreq(n = len(time), d = timestep)

            periods = (1.0 / frequencies)

            # Defines the csv file object.
            csv_file = open(savepath, 'w', encoding = 'utf-8')

            # Initializes the header variable.
            header = 'Period'

            # Generates the header: Time,Planet 1 h,Planet 1 k,Planet 1 p,Planet 2 h...
            for planet_num in range(1, multiplicity + 1):
                planet_label = 'Planet {n}'.format(n = planet_num)
                header += ',' + planet_label + ' h fft,' + planet_label + ' k fft,' + planet_label + ' p fft,' + planet_label + ' q fft'

            # Ends the header line.
            header += '\n'

            # Writes the header to the .csv file.
            csv_file.write(header)

            # Initializes a set of four vector lists of dimension (multiplicity, 1e6).
            h_ffts = np.empty((multiplicity, 5001))
            k_ffts = np.empty((multiplicity, 5001))
            p_ffts = np.empty((multiplicity, 5001))
            q_ffts = np.empty((multiplicity, 5001))

            # Loops through all possible number of planets in a system (1-9 Planets).
            for planet_num in range(multiplicity):
                h_ffts[planet_num] = data
                k_ffts[planet_num] = data
                p_ffts[planet_num] = data
                q_ffts[planet_num] = data

                # Guards against index errors.
                assert len(h_ffts[planet_num]) == len(k_ffts[planet_num]) == len(p_ffts[planet_num]) == len(q_ffts[planet_num])

            h_ffts = h_ffts.T
            k_ffts = k_ffts.T
            p_ffts = p_ffts.T
            q_ffts = q_ffts.T

            # Loops through each planet in the given system.
            for row_num in range(len(h_ffts)):
                # Initializes the row output.
                row = str()

                # Loops through each column of the given row.
                for col_num in range(len(h_ffts[row_num])):
                    h_str = h_ffts[row_num][col_num].astype(str)
                    k_str = k_ffts[row_num][col_num].astype(str)
                    p_str = p_ffts[row_num][col_num].astype(str)
                    q_str = q_ffts[row_num][col_num].astype(str)

                    if col_num == 0:
                        # Gets the value of each vector at the given row index, casting to strings.
                        periods_str = periods[row_num].astype(str)

                        row += periods_str + ','
                    else:
                        row += ','

                    # Generates the output string for the current row.
                    row += ','.join([h_str, k_str, p_str, q_str])

                row += '\n'

                # Writes the row to the .csv file.
                csv_file.write(row)

        # Handles exceptions.
        except FileNotFoundError as e:
            print(e)

In [18]:
input_path = os.path.join(os.pardir, os.pardir, 'data', 'raw')
output_path = os.path.join(os.pardir, os.pardir, 'data', 'processed', 'vectors')
files = os.listdir(input_path)

system_filepaths = [os.path.join(input_path, file) for file in files]
savepaths = [os.path.join(output_path, file) for file in files]

generate_power_spectra_data(savepaths, system_filepaths)

C:\Users\natha\AppData\Local\Temp\ipykernel_18620\201285024.py:33: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_18620\201285024.py:33: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_18620\201285024.py:33: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_18620\201285024.py:33: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_18620\201285024.py:33: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_18620\201285024.py:33: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_18620\201285024.py:33: RuntimeWarning: divid

In [11]:
output_path = os.path.join(os.pardir, os.pardir, 'data', 'processed', 'vectors')
files = ['test100.csv', 'test50.csv', 'test20.csv']

savepaths = [os.path.join(output_path, file) for file in files]

generate_test_data(savepaths, [100, 50, 20], 3)

C:\Users\natha\AppData\Local\Temp\ipykernel_404\935395872.py:26: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_404\935395872.py:26: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)
C:\Users\natha\AppData\Local\Temp\ipykernel_404\935395872.py:26: RuntimeWarning: divide by zero encountered in divide
  periods = (1.0 / frequencies)


In [19]:
'''x = data['Time']
y = data['Planet 1 h'] #['Planet 1 ecc'] * np.sin(data['Planet 1 omega'] + data['Planet 1 Omega'])

fft = np.abs(np.fft.rfft(y))
freq = np.fft.rfftfreq(n=len(y), d=100)

(fig, ax) = plt.subplots(1, 1)

ax.plot(1/freq, fft, rasterized = True)
ax.set_xscale('log')'''

"x = data['Time']\ny = data['Planet 1 h'] #['Planet 1 ecc'] * np.sin(data['Planet 1 omega'] + data['Planet 1 Omega'])\n\nfft = np.abs(np.fft.rfft(y))\nfreq = np.fft.rfftfreq(n=len(y), d=100)\n\n(fig, ax) = plt.subplots(1, 1)\n\nax.plot(1/freq, fft, rasterized = True)\nax.set_xscale('log')"